# Script for Processing Data

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
import random
import glob
import umap
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
# from ggplot import *


from ruffus import *
import sys, os, h5py, random, tempfile, scipy, time,copy
import cmapPy.pandasGEXpress.parse_gctx as parse_gctx
import cmapPy.pandasGEXpress.parse_gct as parse_gct
import pipeline_support as PS
from sklearn.decomposition import PCA
from scipy import stats
from matplotlib.pyplot import imshow
from sklearn.preprocessing import MinMaxScaler

from collections import Counter
#from tensorflow_examples.models.pix2pix import pix2pix
import tensorflow as tf
from tensorflow import keras
from IPython.display import clear_output
from tensorflow_gan.python.losses import losses_impl


from numpy.random import seed
randomState = 123
seed(randomState)

Parameters

In [2]:
n_sampling = 50000

Input Filenames

In [3]:
l1000_filename = "../data/L1000/GSE92742_Broad_LINCS_Level3_INF_mlr12k_n1319138x12328.gctx"
l1000_geneinfo_filename = "../data/L1000/GSE92742_Broad_LINCS_gene_info.txt"

l1000_all_gene_list = "../data/L1000/all_gene_list.txt"
l1000_landmark_gene_list = "../data/L1000/landmark_gene_list.txt"

archs4_all_gene_list = "../data/ARCHS4/all_gene_list.txt"

gtex_l1000_all_gene_list = "../data/GTEx/l1000_all_gene_list.txt"
gtex_l1000_landmark_gene_list = "../data/GTEx/l1000_landmark_gene_list.txt"

gtex_rnaseq_all_gene_list = "../data/GTEx/rnaseq_all_gene_list.txt"

Output Filenames

In [8]:
l1000_output_filename = "../data/processed/L1000/L1000_filtered_GSE92742_Broad_LINCS_Level3_INF_mlr12k_n{}x{}.pkl" # only landmark genes
l1000_overlap_landmark_output_filename = "../data/processed/L1000/L1000_filtered_GSE92742_Broad_LINCS_Level3_INF_mlr12k_n{}x{}.f" # n_samplingx967


## Get overlap_landmark_file

In [6]:
with open(l1000_landmark_gene_list, "r") as f:
    l1000_landmark_gene = [x.strip() for x in f.readlines()]
with open(archs4_all_gene_list, "r") as f:
    archs4_all_gene = [x.strip() for x in f.readlines()]
with open(gtex_l1000_landmark_gene_list, "r") as f:
    gtex_l1000_landmark_gene = [x.strip() for x in f.readlines()]
with open(gtex_rnaseq_all_gene_list, "r") as f:
    gtex_rnaseq_all_gene = [x.strip() for x in f.readlines()]
    



In [7]:
overlap_landmark_genes = list(set(l1000_landmark_gene).intersection(archs4_all_gene).intersection(gtex_l1000_landmark_gene).intersection(gtex_rnaseq_all_gene))

## Load L1000 (GSE92742/Level 3) ~4 min

https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE92742 downloaded @ ../data

In [10]:
print('Loading L1000 data.....')
l1000_data = parse_gctx.parse(l1000_filename,convert_neg_666=True).data_df

Loading L1000 data.....


In [11]:
gene_info = pd.read_csv(l1000_geneinfo_filename,header = 0, sep = '\t')

In [12]:
# create a probe_id to gene name dictionary 
gene_dict = dict(zip([str(x) for x in gene_info['pr_gene_id']], gene_info['pr_gene_symbol']))

# label rows with gene names 
l1000_data.index = [gene_dict[x] for x in l1000_data.index.values]


In [13]:
# filter landmark genes
filtered_l1000_data = l1000_data[l1000_data.index.isin(overlap_landmark_genes)]

In [14]:
# save filtered_l1000 data (only landmark genes)
with open(l1000_output_filename.format(filtered_l1000_data.shape[0], filtered_l1000_data.shape[1]), "wb") as f:
    pickle.dump(filtered_l1000_data)
# filtered_l1000_data.to_csv(l1000_output_filename)

Remove variables

In [ ]:
del l1000_data

In [ ]:
n_sampling = 10000
filtered_l1000_data_sampled = filtered_l1000_data.sample(axis='columns', n=n_sampling).T
filtered_l1000_data_sampled.reset_index().to_feather(l1000_overlap_landmark_output_filename.format(filtered_l1000_data_sampled.shape[0], filtered_l1000_data_sampled.shape[1]))

In [ ]:
filtered_l1000_data_sampled.shape

In [30]:
filtered_l1000_data_sampled

,PSME1,ATF1,RHEB,FOXO3,RHOA,IL1B,ASAH1,RALA,ARHGEF12,SOX2,...,CCP110,STK10,UGDH,SQRDL,HEBP1,ATP11B,CD320,MLLT11,CEBPZ,CBR3
cid,,,,,,,,,,,,,,,,,,,,,
CPC012_HT29_6H_X5_B5_DUO52HI53LO:D04,13.477776,5.01080,14.100801,5.629050,5.4623,7.09685,4.81885,5.99850,4.99910,10.275200,...,4.58325,7.12655,6.047200,4.613700,5.706400,13.440675,8.32390,4.489850,7.55380,4.01465
KDB009_HCC515_96H_X2_F1B5_DUO52HI53LO:G16,8.049150,7.35170,13.616500,4.305300,5.6325,7.21895,7.72420,9.06150,5.96490,8.769199,...,5.66610,8.15330,8.398600,7.417400,9.216600,12.091500,9.78250,5.875900,8.57600,4.91680
ERG012_VCAP_24H_X1.A2_B7_DUO52HI53LO:M15,4.450900,5.85880,12.536100,9.100700,8.3863,4.74540,8.14340,6.45530,6.03810,6.249800,...,5.91165,6.13520,11.021000,6.264350,8.676050,9.594250,11.22700,6.080900,8.89900,8.42610
KDB003_PC3_144H_X1_B1_DUO52HI53LO:H08,10.815250,5.64290,12.041000,7.289199,6.9139,7.26820,6.23385,8.50860,6.11720,5.655500,...,4.86600,5.88330,9.835199,7.363200,9.039600,8.050900,11.61300,7.304700,8.80860,8.82910
KDB010_VCAP_120H_X3_F1B4_DUO52HI53LO:H09,9.947701,6.25790,12.581500,8.076000,8.4675,5.89930,8.68190,8.35270,4.71190,6.206050,...,6.54490,5.79170,10.598000,9.193899,8.640800,10.057800,9.92050,8.906549,7.49850,6.54490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RAD001_MCF7_24H_X3_F1B5_DUO52HI53LO:E11,11.167800,7.73235,12.275000,7.581200,8.3550,4.91600,7.05400,8.86350,4.40450,6.419800,...,9.98920,6.95280,13.540800,5.683300,7.260000,9.754700,9.21900,4.551500,8.85835,5.08900
CYT001_A375_2H_X2_B7_DUO52HI53LO:B07,3.641300,8.37035,12.522800,6.069800,8.1981,5.14050,6.18295,9.39990,6.87260,7.957100,...,8.72190,7.20355,8.364000,5.055450,7.073151,9.340200,12.27170,9.897200,10.03870,7.69300
CVD001_PHH_24H_X3_F1B3_DUO52HI53LO:O21,13.547700,5.76875,12.943200,7.431800,8.5162,7.14735,6.01770,8.77890,4.97155,7.750150,...,5.26060,7.06830,11.671400,9.739000,10.792750,10.680700,8.35130,5.414000,7.57130,6.77060


In [34]:
filtered_l1000_data_sampled["AARS"]

cid
CPC012_HT29_6H_X5_B5_DUO52HI53LO:D04          9.209949
KDB009_HCC515_96H_X2_F1B5_DUO52HI53LO:G16    10.403400
ERG012_VCAP_24H_X1.A2_B7_DUO52HI53LO:M15      9.008100
KDB003_PC3_144H_X1_B1_DUO52HI53LO:H08         9.590100
KDB010_VCAP_120H_X3_F1B4_DUO52HI53LO:H09      9.603950
                                               ...    
RAD001_MCF7_24H_X3_F1B5_DUO52HI53LO:E11      11.564250
CYT001_A375_2H_X2_B7_DUO52HI53LO:B07         10.309200
CVD001_PHH_24H_X3_F1B3_DUO52HI53LO:O21       10.088100
KDD002_HA1E_96H_X1_F2B5_DUO52HI53LO:J04      10.313000
KDD003_MCF7_96H_X3_B4_DUO52HI53LO:C13        12.122601
Name: AARS, Length: 50000, dtype: float32

In [33]:
filtered_l1000_data_sampled.loc[:, overlap_landmark_genes].sort_index(axis=1)

,AARS,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,ACAT2,...,ZMIZ1,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10
cid,,,,,,,,,,,,,,,,,,,,,
CPC012_HT29_6H_X5_B5_DUO52HI53LO:D04,9.209949,5.75080,4.308750,13.440675,4.622875,5.19140,7.18615,6.47970,5.17415,10.9587,...,13.514800,4.995975,7.87965,5.59720,4.91680,15.0000,5.09905,7.02635,8.00165,11.113700
KDB009_HCC515_96H_X2_F1B5_DUO52HI53LO:G16,10.403400,6.45310,4.241800,12.498200,7.515600,5.61990,7.18070,7.70290,5.85860,11.1633,...,10.180600,7.015600,6.70020,7.26735,7.05015,15.0000,7.02680,6.49510,7.06650,8.621300
ERG012_VCAP_24H_X1.A2_B7_DUO52HI53LO:M15,9.008100,9.70340,5.993200,10.855900,6.824400,6.86080,6.70180,9.30120,8.43960,9.6219,...,9.790900,9.381300,8.38970,7.69920,6.86740,12.3333,8.77630,7.75080,7.88865,9.637700
KDB003_PC3_144H_X1_B1_DUO52HI53LO:H08,9.590100,7.94465,3.987100,9.209700,9.610900,8.00420,7.46010,7.27510,9.70110,9.6650,...,10.777349,9.638600,7.60975,7.42100,5.01470,10.3589,7.16915,7.21880,6.62430,9.972300
KDB010_VCAP_120H_X3_F1B4_DUO52HI53LO:H09,9.603950,10.38880,4.646700,11.107000,8.660601,6.59900,6.28720,5.16425,6.45430,5.3859,...,9.398350,9.022350,6.45430,7.41420,7.08405,13.8868,8.35270,11.37980,8.70010,8.768700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RAD001_MCF7_24H_X3_F1B5_DUO52HI53LO:E11,11.564250,10.25535,5.034800,11.748100,11.769800,6.15640,5.93450,6.48730,8.29680,9.2389,...,10.749600,11.998900,9.18330,7.92250,5.63655,13.7482,6.63240,5.71355,5.59950,9.722651
CYT001_A375_2H_X2_B7_DUO52HI53LO:B07,10.309200,8.54990,4.838800,11.140800,8.370350,5.58470,6.25640,9.10400,6.58430,9.2730,...,9.868851,8.608600,8.09070,7.33825,5.82690,11.5249,6.67700,7.05360,7.20355,9.138300
CVD001_PHH_24H_X3_F1B3_DUO52HI53LO:O21,10.088100,9.75205,9.042150,11.943300,8.692300,8.59400,9.22200,7.51105,8.57660,11.9849,...,9.937800,6.865950,6.79795,12.01450,7.72400,13.6431,7.52230,12.67740,7.22620,10.482700
